In [1]:
from typing import List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("juierror/text-to-sql-with-table-schema")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/text-to-sql-with-table-schema")

def prepare_input(question: str, table: List[str]):
    table_prefix = "table:"
    question_prefix = "question:"
    join_table = ",".join(table)
    inputs = f"{question_prefix} {question} {table_prefix} {join_table}"
    input_ids = tokenizer(inputs, max_length=700, return_tensors="pt").input_ids
    return input_ids

def inference(question: str, table: List[str]) -> str:
    input_data = prepare_input(question=question, table=table)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=700)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result

print(inference(question="get people name with age equal 25", table=["id", "name", "age"]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


SELECT name FROM table WHERE age = 25


In [5]:
import pandas as pd
from sqlalchemy.engine import create_engine

In [6]:
predicted_data_path = '/Users/macpro/Documents/GitHub/fuel-electric-hybrid-vehicle-ml/data/predicted-data/vehicle_data_with_clusters.csv'

df = pd.read_csv(predicted_data_path)

df.columns = df.columns.str.replace('.', '_')

# Replace the character '/' with '_per_' all entries
df.columns = df.columns.str.replace('/', '_per_')

# drop column hybrid_in_fuel	hybrid_in_electric	aggregate_levels	vehicle_type_cat
df = df.drop(['hybrid_in_fuel', 'hybrid_in_electric', 'aggregate_levels','transmission_','fuel_type'], axis=1)

/var/folders/2t/nqb9hcfs07n91h4v5p34slp00000gn/T/ipykernel_62938/4036504853.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.', '_')


In [7]:
engine = create_engine("sqlite://")
df = pd.read_csv(predicted_data_path)

In [8]:
df.columns

Index(['vehicle_id', 'vehicleclass_', 'make_', 'model.1_', 'model_year',
       'cylinders_', 'fuelconsumption_city(l/100km)',
       'fuelconsumption_hwy(l/100km)', 'fuelconsumption_comb(l/100km)',
       'co2emissions_(g/km)', 'number_of_gears', 'predicted_co2_rating',
       'enginesize_(l)', 'transmission_', 'fuel_type',
       'fuelconsumption_comb(mpg)', 'smog_rating', 'transmission_type',
       'mapped_fuel_type', 'type_of_wheel_drive', 'vehicle_type', 'motor_(kw)',
       'consumption_combinedle/100km', 'range1_(km)', 'recharge_time(h)',
       'fuel_type2', 'range2_(km)', 'hybrid_fuels',
       'consumption_city(kwh/100km)', 'fuelconsumption_hwy(kwh/100km)',
       'fuelconsumption_comb(kwh/100km)', 'fuelconsumption_city(le/100km)',
       'fuelconsumption_hwy(le/100km)', 'fuelconsumption_comb(le/100km)',
       'range_(km)', 'hybrid_in_fuel', 'hybrid_in_electric',
       'aggregate_levels'],
      dtype='object')

In [9]:
print(inference(question="Show me hybrid car models", table=df.columns))

SELECT hybrid_fuels FROM table WHERE vehicle_type = vehicle_class_ = vehicle_id AND make_ = vehicle_year AND fuelconsumption_comb(l/100km) = fuelconsumption_city(kwh/100km) = fuelconsumption_comb(kwh/100km) = fuel_type = hybrid_car


In [ ]:
%load_ext sql
%sql engine